In [1]:
from datetime import datetime
# Aktuelles Datum und Uhrzeit im ISO-Format holen
timestamp = datetime.now().isoformat(timespec='minutes').replace(":", "")

In [2]:
timestamp

'2025-03-02T1324'

In [3]:
from IPython.display import display
from PIL import Image
import io

In [4]:
from autonomi_client import Client, Wallet, PaymentOption
# Connect to network
client = await Client.init()

In [5]:
from autonomi_client import ChunkAddress, DataMapChunk, Metadata, Network, PublicKey, SecretKey

In [6]:
from autonomi_client import PaymentOption

In [9]:
import py_ecc

In [10]:
data_address = "a7d2fdbb975efaea25b7ebe3d38be4a0b82c1d71e9b89ac4f37bc9f8677826e0"

In [11]:
import time

## interaction with immutable chunks

- the issues i'm having is how to read them because i somehow don't see how i can get a valid address/XOR-Name object

In [13]:
now = time.time()
dog_picture = await client.data_get_public(data_address)

print(f"duration: {time.time()-now} seconds")

TypeError: argument 'addr': 'str' object cannot be converted to 'DataAddress'

In [14]:
data_address = ChunkAddress("a0f6fa2b08e868060fe6e57018e3f73294821feaf3fdcf9cd636ac3d11e7e2ac")

TypeError: argument 'addr': 'str' object cannot be converted to 'XorName'

In [15]:
import autonomi_client as ac

---

neither DataAdddress nor XorName seem to be part of the autonomi_client library

---

In [16]:
[name for name in dir(ac) if isinstance(getattr(ac, name), type)]

['ChunkAddress',
 'Client',
 'ClientConfig',
 'DataMapChunk',
 'GraphEntry',
 'GraphEntryAddress',
 'Metadata',
 'Network',
 'PaymentOption',
 'Pointer',
 'PointerAddress',
 'PointerTarget',
 'PrivateArchive',
 'PublicArchive',
 'PublicKey',
 'RegisterHistory',
 'Scratchpad',
 'ScratchpadAddress',
 'SecretKey',
 'UserData',
 'VaultSecretKey',
 'Wallet']

In [17]:
import getpass

In [18]:
private_key = getpass.getpass('enter private_key')

enter private_key ········


In [19]:
network = Network(False)

In [20]:
wallet = Wallet.new_from_private_key(network, private_key)
#print(f"Wallet address: {wallet.address()}")
print(f"Wallet balance: {await wallet.balance()}")

Wallet balance: 10814818351247818813


In [21]:
await wallet.balance_of_gas()

'44183060000000'

In [22]:
data = b"ants will conquer the world!"

In [23]:
# Choose to pay using our wallet
payment_option = PaymentOption.wallet(wallet)

In [24]:
client = await Client.init()

In [25]:
data_on_network = await client.data_put_public(data, payment_option)

RuntimeError: Failed to put data: Error occurred during payment.

i'm pretty sure the data should already exist anyway - i uploaded it with the previous version of the library if i'm not mistaken

# scratchpads

some experiments with scratchpad

i wanted to deterministically generate scratchpad keys from names ..

In [28]:
from py_ecc.bls import G2ProofOfPossession as bls
from py_ecc.bls12_381 import curve_order
import secrets
import hashlib


def generiere_zufaelliges_bls_schluesselpaar() -> tuple[bytes, bytes]:
    """
    Erzeugt ein zufälliges BLS-Schlüsselpaar.

    :return: Ein Tupel bestehend aus (privater Schlüssel, öffentlicher Schlüssel).
    """
    privater_schluessel: int = secrets.randbelow(curve_order)  # Richtige Kurvenordnung verwenden
    oeffentlicher_schluessel: bytes = bls.SkToPk(privater_schluessel)
    return privater_schluessel.to_bytes(32, "big"), oeffentlicher_schluessel


def ableiten_bls_schluesselpaar_aus_string(eingabe: str) -> tuple[bytes, bytes]:
    """
    Erzeugt deterministisch ein BLS-Schlüsselpaar aus einem String.

    :param eingabe: Der Eingabestring, aus dem der Schlüssel abgeleitet wird.
    :return: Ein Tupel bestehend aus (privater Schlüssel, öffentlicher Schlüssel).
    """
    gehashter_wert: bytes = hashlib.sha256(eingabe.encode()).digest()
    privater_schluessel: int = int.from_bytes(gehashter_wert, "big") % curve_order  # Korrektur hier
    oeffentlicher_schluessel: bytes = bls.SkToPk(privater_schluessel)
    return privater_schluessel.to_bytes(32, "big"), oeffentlicher_schluessel


# Beispielaufrufe
priv, pub = generiere_zufaelliges_bls_schluesselpaar()
print("Zufälliger privater Schlüssel:", priv.hex())
print("Zufälliger öffentlicher Schlüssel:", pub.hex())

det_priv, det_pub = ableiten_bls_schluesselpaar_aus_string("mein geheimer input")
print("Deterministischer privater Schlüssel:", det_priv.hex())
print("Deterministischer öffentlicher Schlüssel:", det_pub.hex())

Zufälliger privater Schlüssel: 30345d003115642502ddd5bb9e456e0348056b6454b870ed7039708f01f60415
Zufälliger öffentlicher Schlüssel: 866bbc1024ebb832974807fec1a176f7a020050f1f1bc56078793b706493444e735990dd188a20551707ff0a58e1d9ba
Deterministischer privater Schlüssel: 43b8c6fb66333ec71f677e6b4f2ff6746d757cf8ed2ea2b196fa6f43c534796e
Deterministischer öffentlicher Schlüssel: a3eb01ce90ca060019b6cd67908baa0a14742094f7072af6dff029e0cad5ad25fc95458ee6149f389f1bae5a03443715


In [29]:
det_priv, det_pub = ableiten_bls_schluesselpaar_aus_string("crappychat")
print("Deterministischer privater Schlüssel:", det_priv.hex())
print("Deterministischer öffentlicher Schlüssel:", det_pub.hex())

Deterministischer privater Schlüssel: 0847a6d42d6090e332ae3169e65debfaf43a4f4e26a998a9b995f6af05b53445
Deterministischer öffentlicher Schlüssel: a060a04b8b5f6bf54b36f66a71310d709e56c445c8a285e15c91a969eb74bd9d048bcf14b33da055caf19e768f2f1fda


In [30]:
mykey = SecretKey.from_hex(det_priv.hex())

In [31]:
mypubkey = PublicKey.from_hex(det_pub.hex())

In [33]:
from autonomi_client import ScratchpadAddress, Scratchpad

In [34]:
address = ScratchpadAddress(mypubkey)

In [35]:
pad = Scratchpad(mykey,7,b'test',7)

In [36]:
pad.decrypt_data(mykey)

b'test'

In [37]:
await client.scratchpad_check_existance(address)

True

In [38]:
await client.scratchpad_put(pad, payment_option)

('0.00000000000000000000000000000000',
 ScratchpadAddress('a060a04b8b5f6bf54b36f66a71310d709e56c445c8a285e15c91a969eb74bd9d048bcf14b33da055caf19e768f2f1fda'))

In [39]:
pad = await client.scratchpad_get(address)

RuntimeError: Failed to get scratchpad: Scratchpad found at ScratchpadAddress(PublicKey(0060..5188)) was not a valid record.

In [40]:
det_priv, det_pub = ableiten_bls_schluesselpaar_aus_string("crappychat_v2")
print("Deterministischer privater Schlüssel:", det_priv.hex())
print("Deterministischer öffentlicher Schlüssel:", det_pub.hex())

Deterministischer privater Schlüssel: 5be65401080eeaf95988e4bba7f2063c386799efc46168acb32b73f5f17ff81a
Deterministischer öffentlicher Schlüssel: a3e2789bc7496fa305439609a6006b14dd5e7ceaa382454ac2a8bb214de264eb56857c56a142b7be4815cc6ce774a6b8


In [41]:
mykey2 = SecretKey.from_hex(det_priv.hex())

In [42]:
mypubkey2 = PublicKey.from_hex(det_pub.hex())

In [43]:
new_scratchpad = await client.scratchpad_create(mykey2,7,b'crappychat',payment_option)

RuntimeError: Failed to create scratchpad: Payment failure occurred during scratchpad creation.

In [44]:
pad = Scratchpad(mykey2,7,b'test',7)

In [45]:
await client.scratchpad_put(pad, payment_option)

RuntimeError: Failed to put scratchpad: Payment failure occurred during scratchpad creation.